In [1]:
from time import sleep

import requests
import json
import pandas as pd
import time
from collections import defaultdict
from datetime import datetime

from dotenv import load_dotenv
import os


In [2]:
load_dotenv(dotenv_path='app.env')

token = os.getenv('GITHUB_TOKEN')

In [3]:
# Read the CSV file
file_path = 'Data/Application_types.csv'
df = pd.read_csv(file_path)

# Convert the 'Keyword' column to a list
topic_list = df['Keyword'].tolist()

# Display the list
print(topic_list)


def write_json_data(fileName, data):
    with open(fileName, 'w') as file:
        json.dump(data, file, indent=4)


def append_json_data(fileName, data):
    # Try to read existing data if the file exists
    try:
        with open(fileName, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []  # Initialize as an empty list if the file doesn't exist

    # Append new data to existing data
    existing_data.append(data)

    # Write the updated data back to the file
    with open(fileName, 'w') as file:
        json.dump(existing_data, file, indent=4)


def sleep_if_rate_exhausted(response):
    core_limits = response.headers._store
    rate_left = core_limits['x-ratelimit-remaining'][1]

    if int(rate_left) > 0:
        return

    reset_time = core_limits['x-ratelimit-reset'][1]
    reset_timestamp = datetime.fromtimestamp(int(reset_time))
    current_time = datetime.now()
    sleep_duration = (reset_timestamp - current_time).total_seconds()
    print(f"Rate limit reached. Sleeping until reset at {reset_timestamp}")
    time.sleep(sleep_duration+5)  # Sleep until reset time + some delay time
    print("Rate limit reset. Resuming requests.")


['Dashboard', 'CMS', 'E-commerce', 'Portfolio', 'Blog', 'Chatbot', 'Social Network', 'Messaging', 'Forum', 'Community', 'News', 'Wiki', 'File Manager', 'Analytics', 'Project Management', 'Task Manager', 'Calendar', 'Notes', 'Kanban', 'ERP', 'CRM', 'POS', 'Inventory', 'To-do', 'Video Streaming', 'Music Player', 'Photo Gallery', 'Weather', 'Finance', 'Budget Tracker', 'Expense Tracker', 'Banking', 'Travel', 'Booking', 'Reservation', 'Hotel Management', 'Real Estate', 'Job Portal', 'Classroom', 'Learning Management System', 'Online Exam', 'Survey', 'Poll', 'E-learning', 'Quiz', 'Recipe', 'Cookbook', 'Portfolio', 'Resume Builder', 'Event Management', 'Gaming', 'Game Tracker', 'Game Leaderboard', 'VR', 'AR', '3D Modeling', 'Fitness', 'Workout Tracker', 'Health Tracker', 'Diet Planner', 'Meal Planner', 'Meditation', 'Habit Tracker', 'Productivity', 'Chat', 'Collaboration', 'Help Desk', 'Customer Support', 'Feedback', 'Documentation', 'Startup', 'Portfolio', 'Map', 'Geo-location', 'Travel Gui

In [21]:
def get_github_projects_of_topic(num_results,num_pages, topic):
    headers = {'Authorization': f'token {token}'}
    url = 'https://api.github.com/search/repositories'
    params = {
        'q': f'(language:JavaScript OR language:TypeScript) {topic} stars:>50',
        'sort': 'stars',
        'order': 'desc',
        'per_page': num_results,
        'page': num_pages
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        # write_json_data('Data/API_Response/topic_data.json', data)
        # pretty_json = json.dumps(data['items'], indent=4)
        # print(pretty_json)
        projects = [{'id': item['id'], 'name': item['name'], 'owner': item['owner']['login'],
                     'watchers': item['watchers_count'],
                     'created_at': item['created_at'], 'updated_at': item['updated_at']
                        , 'size': item['size'], 'stars': item['stargazers_count']
                        , 'open_issues_count': item['open_issues_count']
                        , 'forks': item['forks']} for item in data['items']]
        
        
        
        for project in projects:
            if project['id'] in projects_set:
                continue
            unique_projects.append(project)
            projects_set.add(project['id'])
            if(len(unique_projects) >= total_num_results):
                break

        

        return unique_projects, response,len(unique_projects),len(projects)

    else:
        print(f"Error: {response.status_code}")
        return None, response

topic_dict = {}
num_topics = 40
projects_set = set()
for i in range(num_topics):
    currentTopic = topic_list[i]
    num_unique_results = 0
    num_pages=1
    unique_projects = []
    prev_results=0
    num_results = 10
    total_num_results = num_results
    while True:
        projects, response, num_unique_results, num_of_results_from_API = get_github_projects_of_topic(num_results=num_results, num_pages=num_pages,topic=currentTopic)
        
        if projects is not None:
            topic_dict[currentTopic] = projects
        
        if num_unique_results >= total_num_results:
            break
        
        if num_of_results_from_API == prev_results:
            break
        
        prev_results = num_of_results_from_API
        
        if num_results<100:
            num_results+=10
        else:
            num_pages+=1
        
        
        sleep_if_rate_exhausted(response)
        
    sleep_if_rate_exhausted(response)
    
        
    
        

print(topic_dict)








Rate limit reached. Sleeping until reset at 2024-11-16 18:39:48
Rate limit reset. Resuming requests.
Rate limit reached. Sleeping until reset at 2024-11-16 18:40:55
Rate limit reset. Resuming requests.
Rate limit reached. Sleeping until reset at 2024-11-16 18:42:01
Rate limit reset. Resuming requests.
{'Dashboard': [{'id': 88464704, 'name': 'vue-element-admin', 'owner': 'PanJiaChen', 'watchers': 88000, 'created_at': '2017-04-17T03:35:49Z', 'updated_at': '2024-11-16T08:45:32Z', 'size': 15433, 'stars': 88000, 'open_issues_count': 1355, 'forks': 30475}, {'id': 15111821, 'name': 'grafana', 'owner': 'grafana', 'watchers': 65068, 'created_at': '2013-12-11T15:59:56Z', 'updated_at': '2024-11-16T09:01:14Z', 'size': 1175381, 'stars': 65068, 'open_issues_count': 4230, 'forks': 12145}, {'id': 43441403, 'name': 'strapi', 'owner': 'strapi', 'watchers': 63793, 'created_at': '2015-09-30T15:34:48Z', 'updated_at': '2024-11-16T09:58:40Z', 'size': 529306, 'stars': 63793, 'open_issues_count': 912, 'forks':

In [22]:
all_projects = []

filename = 'github_40_projects_with_topics_3'
for topic, projects in topic_dict.items():
    for project in projects:
        # Add the topic as a new field in each project dictionary
        project['topic'] = topic
        all_projects.append(project)

# Convert the list of all projects with topics into a DataFrame
df = pd.DataFrame(all_projects)

# Save the DataFrame to a CSV file
df.to_csv(f'{filename}.csv', index=False)

print(f"Data saved to {filename}.csv")

error_from = ""

Data saved to github_40_projects_with_topics_3.csv


In [28]:
def get_Sbom_of_projects(repo_df):
    headers = {'Authorization': f'token {token}'}
    url = 'https://api.github.com/repos/'
    owner = repo_df['owner']
    repo = repo_df['name']
    url = url + f'{owner}/{repo}/dependency-graph/sbom'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        # write_json_data('./sbom.json',data)
        packages = data['sbom']['packages']
        # Extract npm packages with name and version
        pkg_dict = defaultdict(dict)
        for package in packages:
            if package['externalRefs'][0]['referenceLocator'].startswith('pkg:npm'):
                try:
                    if len(package['versionInfo']) > 0:
                        if 'versionInfo' in package:
                            if package['name'].startswith('npm:'):
                                sbom_dict[repo][package['name'][4:]] = package['versionInfo']
                                pkg_dict[repo][package['name'][4:]] = package['versionInfo']
                            else:
                                sbom_dict[repo][package['name']] = package['versionInfo']
                                pkg_dict[repo][package['name']] = package['versionInfo']
                except:
                    # print(package)
                    pass
        # append_json_data('Data/40_Projects_1/append_dependencies_40.json', pkg_dict)

    else:
        print(f"Error: {response.status_code}")

    return response


sbom_dict = defaultdict(dict)
for topic, projects in topic_dict.items():
    for project in projects:
        response = get_Sbom_of_projects(project)
        sleep_if_rate_exhausted(response)
    print(f'{topic} done')

write_json_data('Data/40_Unique/append_dependencies_40_3.json', sbom_dict)
# print(sbom_dict)

Dashboard done
CMS done
E-commerce done
Portfolio done
Blog done
Chatbot done
Social Network done
Messaging done
Forum done
Community done
News done
Wiki done
File Manager done
Analytics done
Project Management done
Task Manager done
Calendar done
Notes done
Kanban done
ERP done
CRM done
POS done
Inventory done
To-do done
Video Streaming done
Music Player done
Photo Gallery done
Weather done
Finance done
Budget Tracker done
Expense Tracker done
Banking done
Travel done
Booking done
Reservation done
Hotel Management done
Real Estate done
Job Portal done
Classroom done
Learning Management System done


In [5]:
x = json.load(open("Data/40_Projects/append_dependencies_40_1.json"))

print(x)

KeyboardInterrupt: 